# **1. Demo Básica de Retrieval-Augmented Generation (BasicRAG) con Gemini:**

## ¿Cómo funciona?

En este ejemplo veremos cómo funciona un sistema básico de RAG (Retrieval-Augmented Generation).
La idea principal es:

1. Subir un documento de texto (ejemplo: un archivo con información de física cuántica o historia).

2. Dividir el documento en fragmentos pequeños (chunks).

3. Convertir esos fragmentos en embeddings (vectores numéricos que representan el significado del texto).

4. Guardar los embeddings en una base de datos vectorial (ChromaDB).

5. Hacer una pregunta en lenguaje natural → el sistema busca los fragmentos más relevantes.

6. Gemini responde usando esos fragmentos como contexto, para dar una respuesta más precisa y confiable.

Esto es lo que hace especial a RAG: el modelo no depende solo de su memoria entrenada, sino que consulta información externa que nosotros le damos.

## **Preparación del entorno:**

**Antes de ejecutar el código:**

1. **Obtener tu API Key de Gemini:**

- Entra a 👉 Google AI Studio: https://aistudio.google.com/.

- Crea una clave desde la sección API Keys.

- Copia tu API Key y reemplaza en la línea: **os.environ["GEMINI_API_KEY"] = "TU_API_KEY_AQUI"**


**Este paso solo es necesario si la que está puesta falla, o si cada estudiante quiere usar su propia clave.**

2. **Archivos de prueba disponibles en Drive (Debajo se indican los documentos utilizados en la demo en calse): https://drive.google.com/drive/folders/1uF7-oSMpzSdID2ltQf9NsU5Snf16fHfx?usp=sharing**

- cuantica.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- innovadores.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- historia_Colombia.txt → texto sobre historia de Colombia.

- historia_internet.txt → documento con explicación básica de Einstein.

👉 **Suban uno de estos archivos (o el suyo propio) cuando el código lo pida.**

In [53]:
# ================= DEMO RAG CON FLAN vs FLAN sin RAG vs GEMINI ==================
!pip install chromadb sentence-transformers google-generativeai
!pip install transformers datasets evaluate rouge_score peft accelerate huggingface_hub safetensors
!pip install torch
import os
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
from google.colab import files
from google.colab import userdata
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import evaluate
import torch
import numpy as np
import pandas as pd

In [25]:
# ========= 1. CONFIGURACIÓN DE FLAN =========
model_name='google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [88]:
# ========= 2. CONFIGURACIÓN DE GEMINI =========
# 👉 Paso previo: obtener API Key en https://aistudio.google.com/
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [4]:
# ========= 3. SUBIR ARCHIVO =========
print("Sube un archivo de texto con información (ej: cuantica.txt)")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()


Sube un archivo de texto con información (ej: cuantica.txt)


Saving historia_Colombia.txt to historia_Colombia (1).txt


In [5]:
# ========= 4. FUNCIÓN DE CHUNKING =========
def chunk_text(text, chunk_size=80, overlap=20):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

docs = chunk_text(text, chunk_size=80, overlap=20)

print("Ejemplo de 3 chunks creados:")
for c in docs[:3]:
    print("-", c, "\n")

Ejemplo de 3 chunks creados:
- La independencia de Colombia fue un proceso político y militar que tuvo lugar a comienzos del siglo XIX. Entre 1810 y 1819 se desarrollaron múltiples luchas contra el dominio colonial español. El 20 de julio de 1810 se dio el primer grito de independencia en Santa Fe, marcando el inicio de una serie de movimientos revolucionarios en las provincias. Figuras clave como Simón Bolívar, Francisco de Paula Santander y Antonio Nariño jugaron un papel fundamental en la organización militar y 

- clave como Simón Bolívar, Francisco de Paula Santander y Antonio Nariño jugaron un papel fundamental en la organización militar y política. Uno de los momentos decisivos fue la Campaña Libertadora de 1819, liderada por Simón Bolívar. La batalla de Boyacá, ocurrida el 7 de agosto de 1819, selló la independencia al derrotar al ejército realista. Este triunfo permitió la creación de la Gran Colombia, un proyecto político que unió a los actuales territorios de Colombia, Venez

In [6]:
# ========= 5. CREAR EMBEDDINGS Y BASE VECTORIAL =========
embedder = SentenceTransformer("all-MiniLM-L6-v2")
client = chromadb.Client()
collection = client.get_or_create_collection("physics_chunks")

embeddings = embedder.encode(docs).tolist()
for i, d in enumerate(docs):
    collection.add(documents=[d], embeddings=[embeddings[i]], ids=[str(i)])


In [7]:
# ========= 6. CONSULTA Y RETRIEVAL =========
# 👉 Aquí puedes cambiar la pregunta y experimentar ========================================
# Ejemplos para probar:
# query = "¿Qué explica la teoría de la relatividad?"
# query = "¿Cuál fue un hecho clave en la independencia de Colombia?"
query = "¿Cuál fue un hecho clave en la independencia de Colombia?"
q_embed = embedder.encode([query]).tolist()
results = collection.query(query_embeddings=q_embed, n_results=3)
retrieved_context = " ".join(results['documents'][0])

print("\n🔹 Chunks relevantes recuperados:")
for doc in results['documents'][0]:
    print("-", doc)



🔹 Chunks relevantes recuperados:
- La independencia de Colombia fue un proceso político y militar que tuvo lugar a comienzos del siglo XIX. Entre 1810 y 1819 se desarrollaron múltiples luchas contra el dominio colonial español. El 20 de julio de 1810 se dio el primer grito de independencia en Santa Fe, marcando el inicio de una serie de movimientos revolucionarios en las provincias. Figuras clave como Simón Bolívar, Francisco de Paula Santander y Antonio Nariño jugaron un papel fundamental en la organización militar y
- de la Gran Colombia, un proyecto político que unió a los actuales territorios de Colombia, Venezuela, Ecuador y Panamá. Sin embargo, la Gran Colombia enfrentó problemas internos y conflictos entre caudillos regionales. Finalmente, en 1831, la unión se disolvió y cada país tomó su propio rumbo político. La independencia dejó profundas huellas en la identidad colombiana y sentó las bases de la construcción de la nación.
- clave como Simón Bolívar, Francisco de Paula Sant

In [82]:
# ========= 7. FLAN PARA RESPONDER CON CONTEXTO =========

prompt = f"Basándote en el siguiente contexto, responde la pregunta y añade una breve explicación adicional desde tu conocimiento si es relevante.\n\nContexto:\n{retrieved_context}\n\nPregunta: {query}"

inputs = tokenizer(str(prompt), return_tensors='pt')
flan_context = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=10000,
    )[0],
    skip_special_tokens=True
)

print("\n🔹 Respuesta generada por Flan:")
print(flan_context)


🔹 Respuesta generada por Flan:
Simón Bolvar, Francisco de Paula Santander y Antonio Nario jugaron un papel fundamental en la organización militar y de la Gran Colombia


In [83]:
# ========= 8. FLAN PARA RESPONDER SIN CONTEXTO =========

prompt = f"Responde la pregunta y añade una breve explicación adicional desde tu conocimiento si es relevante. \n\nPregunta: {query}"

inputs = tokenizer(str(prompt), return_tensors='pt')
flan_base = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=10000,
    )[0],
    skip_special_tokens=True
)

print("\n🔹 Respuesta generada por Flan:")
print(flan_base)


🔹 Respuesta generada por Flan:
Qué es un hecho clave en la independencia de Colombia?


In [89]:
# ========= 9. GEMINI PARA RESPONDER CON CONTEXTO =========
model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

gemini_context = model.generate_content(
    f"Basándote en el siguiente contexto, responde la pregunta y añade una breve explicación adicional desde tu conocimiento si es relevante.\n\nContexto:\n{retrieved_context}\n\nPregunta: {query}"
)

print("\n🔹 Respuesta generada por Gemini:")
print(gemini_context.text)


🔹 Respuesta generada por Gemini:
Un hecho clave en la independencia de Colombia fue la Batalla de Boyacá, el 7 de agosto de 1819.  Esta batalla, ganada por el ejército independentista liderado por Simón Bolívar, significó una derrota decisiva para las fuerzas realistas españolas y abrió el camino para la creación de la Gran Colombia.  

**Explicación adicional:**  Si bien el Grito de Independencia del 20 de julio de 1810 marcó el inicio del proceso, la Batalla de Boyacá representó un punto de inflexión militar que consolidó las victorias independentistas y aseguró, en la práctica, el fin del dominio español en una gran parte del territorio que luego conformaría la Gran Colombia.  Fue una victoria estratégica que permitió a Bolívar avanzar hacia Bogotá y asegurar la independencia de Nueva Granada (actual Colombia).  Sin la victoria en Boyacá, el camino hacia la independencia habría sido considerablemente más largo y complejo.



In [90]:
# ========= 10. GEMINI PARA RESPONDER SIN CONTEXTO =========
model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

gemini_base = model.generate_content(
    f"Responde la pregunta y añade una breve explicación adicional desde tu conocimiento si es relevante. \n\nPregunta: {query}"
)

print("\n🔹 Respuesta generada por Gemini:")
print(gemini_base.text)


🔹 Respuesta generada por Gemini:
Un hecho clave en la independencia de Colombia fue **la Batalla de Boyacá, librada el 7 de agosto de 1819**.  Esta batalla, una victoria decisiva del ejército patriota liderado por Simón Bolívar sobre las fuerzas realistas españolas, aseguró la independencia del Virreinato de Nueva Granada (que incluía el territorio de la actual Colombia, junto a partes de Venezuela, Ecuador y Panamá).  La derrota española en Boyacá abrió el camino para la liberación de Bogotá y, posteriormente, la declaración formal de la independencia.

**Explicación adicional:** Si bien hubo muchos otros eventos importantes en el proceso independentista (como la Campaña Libertadora de los Andes,  las luchas de precursores como Camilo Torres y Antonio Nariño), la Batalla de Boyacá es considerada clave por su carácter definitorio.  Representó un punto de inflexión estratégico, rompiendo el poderío realista en la Nueva Granada y allanando el camino para la consolidación de la independe

Para la evaluación de métricas se tomará como Y real el resultado de realizar la misma pregunta a gpt 5 con la opción de piensa más tiempo y contexto

In [92]:
chat_response = '''
Un hecho clave fue la Batalla de Boyacá, ocurrida el 7 de agosto de 1819.

Breve explicación: ese triunfo, parte de la Campaña Libertadora dirigida por Simón Bolívar, derrotó al ejército realista y aseguró el camino hacia la independencia efectiva del territorio, permitiendo luego la creación de la Gran Colombia y consolidando el proceso militar y político de emancipación.
'''

In [66]:
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

In [93]:
rouge_flan_context = rouge.compute(
    predictions=[flan_context],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_flan_context = bleu.compute(
    predictions=[flan_context],
    references=[chat_response],
)

rouge_flan_base = rouge.compute(
    predictions=[flan_base],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_flan_base = bleu.compute(
    predictions=[flan_base],
    references=[chat_response],
)

rouge_gemini_context = rouge.compute(
    predictions=[gemini_context.text],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_gemini_context = bleu.compute(
    predictions=[gemini_context.text],
    references=[chat_response],
)

rouge_gemini_base = rouge.compute(
    predictions=[gemini_base.text],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_gemini_base = bleu.compute(
    predictions=[gemini_base.text],
    references=[chat_response],
)

In [98]:
def compare_bleu(result1, result2, name1="Model 1", name2="Model 2"):
    print(f"\n🔎 Comparando {name1} vs {name2}\n")

    # BLEU directo
    print(f"{name1} BLEU: {result1['bleu']}")
    print(f"{name2} BLEU: {result2['bleu']}")

    # Precisions
    for i in range(4):
        print(f"Precisión {i+1}-gram: {name1}={result1['precisions'][i]:.3f} | {name2}={result2['precisions'][i]:.3f}")

    # Longitud
    print(f"\nLongitud traducción: {name1}={result1['translation_length']} | {name2}={result2['translation_length']}")
    print(f"Referencia: {result1['reference_length']} (idéntica para ambos)")

    # Brevity penalty
    print(f"\nBrevity penalty: {name1}={result1['brevity_penalty']:.4f} | {name2}={result2['brevity_penalty']:.4f}")

    # Ranking simple
    context_score = sum(result1['precisions'])
    base_score = sum(result2['precisions'])

    if context_score > base_score:
        print(f"\n🏆 {name1} tiene mejores coincidencias de n-gramas.")
    else:
        print(f"\n🏆 {name2} tiene mejores coincidencias de n-gramas.")

    if result1['translation_length'] > result2['translation_length']:
        print(f"📏 {name1} es más largo y cubre más de la referencia.")
    else:
        print(f"📏 {name2} es más largo y cubre más de la referencia.")

In [101]:
def compare_rouge(result1, result2, name1="Modelo 1", name2="Modelo 2"):
    print(f"\n🔎 Comparando ROUGE entre {name1} y {name2}\n")

    metrics = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

    for m in metrics:
        score1 = float(result1[m])
        score2 = float(result2[m])
        mejor = name1 if score1 > score2 else name2 if score2 > score1 else "Empate"

        print(f"{m.upper():<8} | {name1}: {score1:.3f} | {name2}: {score2:.3f} → 🏆 {mejor}")

    # Promedio general
    avg1 = sum(float(result1[m]) for m in metrics) / len(metrics)
    avg2 = sum(float(result2[m]) for m in metrics) / len(metrics)

    print("\n📊 Promedio de métricas:")
    print(f"{name1}: {avg1:.3f}")
    print(f"{name2}: {avg2:.3f}")

    if avg1 > avg2:
        print(f"\n✅ {name1} tiene mejor solapamiento global con la referencia.")
    elif avg2 > avg1:
        print(f"\n✅ {name2} tiene mejor solapamiento global con la referencia.")
    else:
        print("\n⚖️ Ambos modelos tienen un desempeño similar.")

In [102]:
print("Cambios al incluir CONTEXTO para FLAN rouge")
compare_rouge(rouge_flan_context, rouge_flan_base, "FLAN CONTEXTO", "FLAN BASE")

Cambios al incluir CONTEXTO para FLAN rouge

🔎 Comparando ROUGE entre FLAN CONTEXTO y FLAN BASE

ROUGE1   | FLAN CONTEXTO: 0.286 | FLAN BASE: 0.182 → 🏆 FLAN CONTEXTO
ROUGE2   | FLAN CONTEXTO: 0.112 | FLAN BASE: 0.080 → 🏆 FLAN CONTEXTO
ROUGEL   | FLAN CONTEXTO: 0.198 | FLAN BASE: 0.182 → 🏆 FLAN CONTEXTO
ROUGELSUM | FLAN CONTEXTO: 0.220 | FLAN BASE: 0.182 → 🏆 FLAN CONTEXTO

📊 Promedio de métricas:
FLAN CONTEXTO: 0.204
FLAN BASE: 0.156

✅ FLAN CONTEXTO tiene mejor solapamiento global con la referencia.


In [103]:
compare_bleu(bleu_flan_context, bleu_flan_base, "FLAN CONTEXTO", "FLAN BASE")


🔎 Comparando FLAN CONTEXTO vs FLAN BASE

FLAN CONTEXTO BLEU: 0.021995247780975908
FLAN BASE BLEU: 0.0
Precisión 1-gram: FLAN CONTEXTO=0.478 | FLAN BASE=0.545
Precisión 2-gram: FLAN CONTEXTO=0.182 | FLAN BASE=0.200
Precisión 3-gram: FLAN CONTEXTO=0.095 | FLAN BASE=0.000
Precisión 4-gram: FLAN CONTEXTO=0.050 | FLAN BASE=0.000

Longitud traducción: FLAN CONTEXTO=23 | FLAN BASE=11
Referencia: 66 (idéntica para ambos)

Brevity penalty: FLAN CONTEXTO=0.1542 | FLAN BASE=0.0067

🏆 FLAN CONTEXTO tiene mejores coincidencias de n-gramas.
📏 FLAN CONTEXTO es más largo y cubre más de la referencia.


In [104]:
compare_rouge(rouge_gemini_context, rouge_gemini_base, "GEMINI CONTEXTO", "GEMINI BASE")


🔎 Comparando ROUGE entre GEMINI CONTEXTO y GEMINI BASE

ROUGE1   | GEMINI CONTEXTO: 0.464 | GEMINI BASE: 0.373 → 🏆 GEMINI CONTEXTO
ROUGE2   | GEMINI CONTEXTO: 0.261 | GEMINI BASE: 0.203 → 🏆 GEMINI CONTEXTO
ROUGEL   | GEMINI CONTEXTO: 0.312 | GEMINI BASE: 0.261 → 🏆 GEMINI CONTEXTO
ROUGELSUM | GEMINI CONTEXTO: 0.366 | GEMINI BASE: 0.343 → 🏆 GEMINI CONTEXTO

📊 Promedio de métricas:
GEMINI CONTEXTO: 0.351
GEMINI BASE: 0.295

✅ GEMINI CONTEXTO tiene mejor solapamiento global con la referencia.


In [105]:
compare_bleu(bleu_gemini_context, bleu_gemini_base, "GEMINI CONTEXTO", "GEMINI BASE")


🔎 Comparando GEMINI CONTEXTO vs GEMINI BASE

GEMINI CONTEXTO BLEU: 0.14972460673717317
GEMINI BASE BLEU: 0.07065330249551115
Precisión 1-gram: GEMINI CONTEXTO=0.309 | GEMINI BASE=0.227
Precisión 2-gram: GEMINI CONTEXTO=0.168 | GEMINI BASE=0.095
Precisión 3-gram: GEMINI CONTEXTO=0.119 | GEMINI BASE=0.048
Precisión 4-gram: GEMINI CONTEXTO=0.082 | GEMINI BASE=0.024

Longitud traducción: GEMINI CONTEXTO=162 | GEMINI BASE=211
Referencia: 66 (idéntica para ambos)

Brevity penalty: GEMINI CONTEXTO=1.0000 | GEMINI BASE=1.0000

🏆 GEMINI CONTEXTO tiene mejores coincidencias de n-gramas.
📏 GEMINI BASE es más largo y cubre más de la referencia.


Podemos observar como los modelos con contexto se impusieron sobre los básicos

# 2. **GraphRAG con Gemini + Neo4j: Consultando Grafos de Conocimiento:**

En este ejemplo usamos Gemini para extraer triples semánticos del texto (por ejemplo: (Elon Musk, fundó, Tesla)).

Luego esos triples se guardan en Neo4j, una base de datos orientada a grafos.
Después, podemos hacer consultas usando Cypher, el lenguaje de Neo4j, y finalmente Gemini genera una respuesta en lenguaje natural usando la información consultada.

Esto permite transformar un texto plano en un grafo de conocimiento consultable, lo que es muy útil para preguntas complejas que requieren relaciones entre entidades.

## **Preparación del entorno:**

Antes de ejecutar el código:

1. **Cuenta en Neo4j AuraDB (gratis): https://neo4j.com/product/auradb/**

- Ir a Neo4j AuraDB Free.

- Crear una cuenta y una base de datos gratuita.

- Copiar los datos de conexión (URI, Usuario, Contraseña).

**⚠️ Este paso, al igual que la creación de la API Key de Gemini, solo será necesario si las credenciales ya incluidas en el código no funcionan.
De esta manera, cada estudiante tendrá la opción de usar sus propias credenciales y su propia base de datos personalizada en caso de que sea necesario.**

2. **Nuevamente tener presente los archivos disponibles en Drive:https://drive.google.com/drive/folders/1uF7-oSMpzSdID2ltQf9NsU5Snf16fHfx?usp=sharing**

- cuantica.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- innovadores.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- historia_Colombia.txt → texto sobre historia de Colombia.

- historia_internet.txt → documento con explicación básica de Einstein.

👉 **Suban uno de estos archivos (o el suyo propio) cuando el código lo pida.****

## **Snippets de Cypher para practicar**

Los estudiantes pueden reemplazar el query y la pregunta con estos ejemplos (COPIAR Y PEGAR EN EL BLOQUE DE CODIGO DEL GRAPHRAG):

### **Para el documento innovadores.txt disponible en google drive:** ¿Cuáles son las empresas fundadas por Elon Musk y por Steve Jobs, a qué se dedica cada una y por qué es importante lo que hacen?


In [106]:
cypher_query = """
MATCH (p:Entidad)-[r:RELACION]->(c:Entidad)
RETURN p.name AS persona, r.tipo AS relacion, c.name AS compania
"""

### **Para el documento hisotria_internet.txt disponible en google drive:** ¿Cuál fue la importancia de Tim Berners-Lee en la historia de Internet?

In [107]:
cypher_query = """
MATCH (a:Entidad)-[r:RELACION]->(b:Entidad)
WHERE a.name = "Tim Berners-Lee"
RETURN a.name AS a, r.tipo AS relacion, b.name AS b
"""

In [108]:
# ================== INSTALACIÓN ==================
!pip install neo4j google-generativeai

import os
from neo4j import GraphDatabase
import google.generativeai as genai
from google.colab import files

# 👉 Configura Neo4j con tus propios datos (SOLO si falla la configuración por defecto del Colab)
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USER = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# ================== SUBIR DOCUMENTO ==================
print("Sube un archivo de texto con información (ej: innovadores.txt)")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

# ================== EXTRAER TRIPLES CON GEMINI ==================
model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

prompt = f"""
Extrae relaciones del siguiente texto en formato de triples:
(SUJETO, RELACIÓN, OBJETO).
Texto:
{text}
"""

response = model.generate_content(prompt)
print("🔹 Triples extraídos por Gemini:")
print(response.text)

# ================== GUARDAR TRIPLES EN NEO4J ==================
def insert_triple(tx, s, r, o):
    query = """
    MERGE (a:Entidad {name: $s})
    MERGE (b:Entidad {name: $o})
    MERGE (a)-[rel:RELACION {tipo: $r}]->(b)
    """
    tx.run(query, s=s, r=r, o=o)

triples = []
for line in response.text.split("\n"):
    if "(" in line and ")" in line:
        line = line.strip("()")
        parts = [p.strip() for p in line.split(",")]
        if len(parts) == 3:
            triples.append(parts)

with driver.session() as session:
    for s, r, o in triples:
        session.write_transaction(insert_triple, s, r, o)

print(f"✅ Se insertaron {len(triples)} triples en Neo4j")

# ================== CONSULTA AL GRAFO ==================
def query_graph(query):
    with driver.session() as session:
        result = session.run(query)
        return [dict(r) for r in result]

# AQUI PUEDEN CAMBIAR LAS QUERY POR LOS EJEMPLOS DE ARRIBA ============================== ⏰⏰⏰ =============================
cypher_query = """
MATCH (p:Entidad)-[r:RELACION]->(c:Entidad)
RETURN p.name AS persona, r.tipo AS relacion, c.name AS compania
"""

results = query_graph(cypher_query)

print("\n🔹 Resultados de la consulta Cypher:")
for r in results:
    print(f"{r['persona']} {r['relacion']} {r['compania']}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 8.3 MB/s eta 0:00:00
Sube un archivo de texto con información (ej: innovadores.txt)


Saving innovadores.txt to innovadores.txt
🔹 Triples extraídos por Gemini:
Aquí hay algunas relaciones extraídas del texto en formato de triples (SUJETO, RELACIÓN, OBJETO):

* (Elon Musk, fundó, Tesla)
* (Tesla, tipo_de_compañía, fabricante de automóviles eléctricos)
* (Tesla, se_dedica_a, soluciones de energía renovable)
* (Elon Musk, fundó, SpaceX)
* (SpaceX, tipo_de_empresa, empresa de exploración espacial)
* (SpaceX, desarrolla, cohetes)
* (SpaceX, desarrolla, satélites)
* (Elon Musk, participó_en_la_creación_de, PayPal)
* (PayPal, tipo_de_sistema, sistema de pagos en línea)
* (PayPal, revolucionó, transacciones digitales)
* (Steve Jobs, fue_cofundador_de, Apple)
* (Apple, reconocida_por, productos tecnológicos innovadores)
* (Apple, desarrolló, iPhone)
* (Apple, desarrolló, iPad)
* (Apple, desarrolló, MacBook)
* (Apple, clave_en_el_desarrollo_de, industria de los dispositivos inteligentes)
* (Jeff Bezos, fundó, Amazon)
* (Amazon, inicialmente, librería en línea)
* (Amazon, evolucio

/tmp/ipython-input-3244541476.py:57: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_triple, s, r, o)


✅ Se insertaron 34 triples en Neo4j

🔹 Resultados de la consulta Cypher:
Aquí tienes las relaciones extraídas del texto en formato de triples (SUJETO RELACIÓN OBJETO):
Aquí hay algunas relaciones extraídas del texto en formato de triples (SUJETO RELACIÓN OBJETO):
* (Elon Musk fundó Tesla
* (Tesla tipo_de_compañía fabricante de automóviles eléctricos
* (Tesla se_dedica_a soluciones de energía renovable
* (Elon Musk fundó SpaceX
* (SpaceX tipo_de_empresa empresa de exploración espacial
* (SpaceX desarrolla cohetes
* (SpaceX desarrolla satélites
* (Elon Musk participó_en_la_creación_de PayPal
* (PayPal tipo_de_sistema sistema de pagos en línea
* (PayPal revolucionó transacciones digitales
* (Steve Jobs cofundador_de Apple
* (Steve Jobs fue_cofundador_de Apple
* (Apple reconocida_por productos tecnológicos innovadores
* (Apple produjo iPhone
* (Apple desarrolló iPhone
* (Apple produjo iPad
* (Apple desarrolló iPad
* (Apple produjo MacBook
* (Apple desarrolló MacBook
* (Apple clave_en_el_de

In [109]:
context = "\n".join([f"{r['persona']} {r['relacion']} {r['compania']}" for r in results])

Ya no se usará flan por el tiempo de entrenamiento

In [120]:
# ================== GEMINI CON CONTEXTO PARA RESPUESTA FINAL ==================
context = "\n".join([f"{r['persona']} {r['relacion']} {r['compania']}" for r in results])

final_prompt = f"""
Basándote en el siguiente contexto de un grafo de conocimiento, responde la pregunta y proporciona contexto:

Contexto:
{context}

Pregunta:  ¿Cuáles son las empresas fundadas por Elon Musk y por Steve Jobs, a qué se dedica cada una y por qué es importante lo que hacen?
"""

gemini_context = model.generate_content(final_prompt)
print("\n🔹 Respuesta generada por Gemini con contexto:")
print(gemini_context.text)


🔹 Respuesta generada por Gemini con contexto:
Elon Musk fundó Tesla y SpaceX.

* **Tesla:** Es una fabricante de automóviles eléctricos y se dedica a soluciones de energía renovable.  Su importancia radica en su contribución a la lucha contra el cambio climático a través del desarrollo de vehículos eléctricos y soluciones de energía limpia.

* **SpaceX:** Es una empresa de exploración espacial que desarrolla cohetes y satélites. Su importancia reside en su avance en la tecnología espacial, abriendo nuevas posibilidades para la exploración del espacio y el desarrollo de infraestructura espacial.


Steve Jobs fue cofundador de Apple.

* **Apple:** Es una empresa reconocida por sus productos tecnológicos innovadores, como el iPhone, el iPad y el MacBook.  Su importancia radica en su influencia transformadora en la industria de los dispositivos inteligentes, estableciendo estándares de diseño, usabilidad y experiencia de usuario que han impactado globalmente.  Su desarrollo de productos h

In [121]:
# ================== GEMINI SIN CONTEXTO PARA RESPUESTA FINAL ==================

final_prompt = f"""
Pregunta:  ¿Cuáles son las empresas fundadas por Elon Musk y por Steve Jobs, a qué se dedica cada una y por qué es importante lo que hacen?
"""

gemini_base = model.generate_content(final_prompt)
print("\n🔹 Respuesta generada por Gemini sin contexto:")
print(gemini_base.text)


🔹 Respuesta generada por Gemini sin contexto:
## Empresas fundadas por Elon Musk y Steve Jobs:

**Elon Musk:**

* **SpaceX:** Se dedica a la fabricación de cohetes y naves espaciales reutilizables, con el objetivo de reducir los costos del acceso al espacio y facilitar la colonización de Marte.  Su importancia radica en el potencial de revolucionar la industria espacial, haciendola más accesible y abriendo posibilidades para la exploración espacial a gran escala y la búsqueda de recursos fuera de la Tierra.  Esto podría impactar significativamente en la ciencia, la tecnología y la economía global a largo plazo.

* **Tesla, Inc.:** Se enfoca en el diseño, desarrollo, fabricación y venta de vehículos eléctricos, baterías de energía, paneles solares y sistemas de energía solar. La importancia de Tesla reside en su liderazgo en la transición hacia una energía limpia y sostenible,  impulsando la adopción masiva de vehículos eléctricos y el desarrollo de tecnologías de energía renovable par

In [126]:
chat_response = '''
# Elon Musk — empresas, a qué se dedican y por qué importan

* **Tesla** — fabricante de automóviles eléctricos y proveedor de soluciones de energía renovable.
  **Por qué importa:** impulsa la electrificación del transporte y la adopción de energías limpias, reduciendo emisiones y acelerando la transición hacia sistemas de movilidad y energía más sostenibles.

* **SpaceX** — empresa de exploración espacial que desarrolla cohetes y satélites.
  **Por qué importa:** ha abaratado y acelerado el acceso al espacio (reutilización de cohetes), habilita comunicaciones por satélite y abre posibilidades para la investigación y la exploración interplanetaria.

* **PayPal** (participó en su creación) — sistema de pagos en línea que transformó las transacciones digitales.
  **Por qué importa:** facilitó pagos electrónicos seguros y escalables, impulsando el comercio electrónico y la economía digital.

# Steve Jobs — empresas, a qué se dedican y por qué importan

* **Apple** (cofundador) — compañía tecnológica reconocida por productos como iPhone, iPad y MacBook; desarrolla dispositivos y ecosistemas de software/hardware.
  **Por qué importa:** redefinió la experiencia de usuario en dispositivos personales, impulsó la era del smartphone y creó un ecosistema que influye en diseño, interfaces y modelos de negocio tecnológicos a nivel global.

# Contexto (extraído del grafo)

Las respuestas se basan en las relaciones del grafo provisto: `(Elon Musk fundó Tesla)`, `(Tesla tipo_de_compañía fabricante de automóviles eléctricos)`, `(Tesla se_dedica_a soluciones de energía renovable)`, `(Elon Musk fundó SpaceX)`, `(SpaceX tipo_de_empresa empresa de exploración espacial)`, `(SpaceX desarrolla cohetes)`, `(Elon Musk participó_en_la_creación_de PayPal)`, `(PayPal tipo_de_sistema sistema de pagos en línea)`, `(Steve Jobs cofundador_de Apple)`, `(Apple produjo iPhone/iPad/MacBook)` y `(Apple reconocida_por productos tecnológicos innovadores)`.

Si quieres, puedo ordenar esto en una tabla compacta o añadir ejemplos concretos (p. ej. modelos de Tesla o hitos de SpaceX) — dime si quieres más detalle.
'''

In [127]:
rouge_gemini_context = rouge.compute(
    predictions=[gemini_context.text],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_gemini_context = bleu.compute(
    predictions=[gemini_context.text],
    references=[chat_response],
)

rouge_gemini_base = rouge.compute(
    predictions=[gemini_base.text],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_gemini_base = bleu.compute(
    predictions=[gemini_base.text],
    references=[chat_response],
)

In [128]:
compare_rouge(rouge_gemini_context, rouge_gemini_base, "GEMINI CONTEXTO", "GEMINI BASE")


🔎 Comparando ROUGE entre GEMINI CONTEXTO y GEMINI BASE

ROUGE1   | GEMINI CONTEXTO: 0.434 | GEMINI BASE: 0.344 → 🏆 GEMINI CONTEXTO
ROUGE2   | GEMINI CONTEXTO: 0.200 | GEMINI BASE: 0.121 → 🏆 GEMINI CONTEXTO
ROUGEL   | GEMINI CONTEXTO: 0.260 | GEMINI BASE: 0.170 → 🏆 GEMINI CONTEXTO
ROUGELSUM | GEMINI CONTEXTO: 0.389 | GEMINI BASE: 0.302 → 🏆 GEMINI CONTEXTO

📊 Promedio de métricas:
GEMINI CONTEXTO: 0.321
GEMINI BASE: 0.234

✅ GEMINI CONTEXTO tiene mejor solapamiento global con la referencia.


In [129]:
compare_rouge(rouge_gemini_context, rouge_gemini_base, "GEMINI CONTEXTO", "GEMINI BASE")


🔎 Comparando ROUGE entre GEMINI CONTEXTO y GEMINI BASE

ROUGE1   | GEMINI CONTEXTO: 0.434 | GEMINI BASE: 0.344 → 🏆 GEMINI CONTEXTO
ROUGE2   | GEMINI CONTEXTO: 0.200 | GEMINI BASE: 0.121 → 🏆 GEMINI CONTEXTO
ROUGEL   | GEMINI CONTEXTO: 0.260 | GEMINI BASE: 0.170 → 🏆 GEMINI CONTEXTO
ROUGELSUM | GEMINI CONTEXTO: 0.389 | GEMINI BASE: 0.302 → 🏆 GEMINI CONTEXTO

📊 Promedio de métricas:
GEMINI CONTEXTO: 0.321
GEMINI BASE: 0.234

✅ GEMINI CONTEXTO tiene mejor solapamiento global con la referencia.


# 3. **FusionRAG (BM25 + Embeddings)**

En este ejemplo se combina lo mejor de dos enfoques de recuperación de información:

- BM25 (keyword-based): Recupera pasajes basándose en la coincidencia de palabras clave.

- Vector Search (embeddings con ChromaDB): Recupera fragmentos usando similitud semántica.

Ambos resultados se fusionan para obtener un contexto más robusto y completo, que luego se pasa al modelo Gemini para generar una respuesta.

👉 Deben subir un archivo de texto (ej: cuantica.txt, historia_colombia.txt, innovadores.txt o historia_internet.txt) y luego probar con diferentes preguntas modificando el campo query.

## **Preparación del entorno:**

Si ya configuraste los entornos de los ejemplos anteriores, no necesitas hacer nada adicional para este bloque. 🚀

In [131]:
# ================= DEMO FUSION RAG ==================
!pip install rank_bm25 chromadb sentence-transformers google-generativeai

from rank_bm25 import BM25Okapi
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
import os
from google.colab import files

# ================= CONFIGURACIÓN GEMINI ==================
# 👉 API Key de Gemini (puede usar la nuestra o crear la suya en https://aistudio.google.com/)
model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

# ================= SUBIR DOCUMENTO ==================
print("📂 Sube un archivo de texto con información (ej: cuantica.txt, historia_colombia.txt, etc.)")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

# ================= CHUNKING ==================
def chunk_text(text, chunk_size=80, overlap=20):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]

docs = chunk_text(text)

# ========= Vector Store (embeddings con ChromaDB)
embedder = SentenceTransformer("all-MiniLM-L6-v2")
client = chromadb.Client()
collection = client.get_or_create_collection("fusion_chunks")
embeddings = embedder.encode(docs).tolist()
for i, d in enumerate(docs):
    collection.add(documents=[d], embeddings=[embeddings[i]], ids=[str(i)])

# ========= BM25 retriever
tokenized_corpus = [d.split(" ") for d in docs]
bm25 = BM25Okapi(tokenized_corpus)

# ========= Fusion Retrieval
query = "¿Qué científico propuso un modelo atómico en 1913?"
q_embed = embedder.encode([query]).tolist()
results_vector = collection.query(query_embeddings=q_embed, n_results=3)
results_bm25 = bm25.get_top_n(query.split(" "), docs, n=3)

# Fusión (simple: concatenación + eliminación de duplicados)
fusion_results = list(set(results_vector['documents'][0] + results_bm25))
retrieved_context = " ".join(fusion_results)


📂 Sube un archivo de texto con información (ej: cuantica.txt, historia_colombia.txt, etc.)


Saving cuantica.txt to cuantica.txt


In [132]:
# ================= GEMINI CON CONTEXTO PARA RESPUESTA FINAL ==================
gemini_context = model.generate_content(
    f"Basándote en el siguiente contexto (resultado de fusión de múltiples recuperadores), responde la pregunta y añade explicación:\n\n{retrieved_context}\n\nPregunta: {query}"
)

print("\n🔹 Respuesta generada con FusionRAG:")
print(gemini_context.text)


🔹 Respuesta generada con FusionRAG:
Niels Bohr propuso un modelo atómico en 1913.  El texto lo indica explícitamente: "Niels Bohr en 1913 propuso su modelo atómico, donde los electrones orbitaban en niveles de energía cuantizados alrededor del núcleo".  Este modelo fue un avance significativo en la comprensión de la estructura atómica, introduciendo la idea de niveles de energía discretos para los electrones.



In [133]:
# ================== GEMINI SIN CONTEXTO PARA RESPUESTA FINAL ==================
gemini_base = model.generate_content(
    f"Responde la pregunta y añade explicación:\n\n{retrieved_context}\n\nPregunta: {query}"
)

print("\n🔹 Respuesta generada con FusionRAG:")
print(gemini_base.text)


🔹 Respuesta generada con FusionRAG:
Niels Bohr propuso un modelo atómico en 1913.  Su modelo postulaba que los electrones orbitan el núcleo atómico en niveles de energía cuantizados.  Esto significaba que los electrones sólo podían ocupar ciertas órbitas específicas, a diferencia de los modelos anteriores que permitían órbitas a cualquier distancia del núcleo.  Este concepto de cuantización de la energía fue un paso crucial en el desarrollo de la mecánica cuántica y explicaba, por ejemplo, el espectro de emisión del átomo de hidrógeno.  El modelo de Bohr, aunque posteriormente reemplazado por modelos más precisos, fue fundamental para comprender la estructura atómica y sentó las bases para desarrollos posteriores en la física cuántica.



In [135]:
chat_response = '''
Niels Bohr propuso en 1913 su modelo atómico.

Explicación breve: el modelo de Bohr planteó que los electrones orbitan el núcleo en niveles de energía cuantizados (órbitas discretas) y que los saltos entre esos niveles producen las líneas espectrales —esto ayudó a explicar el espectro del hidrógeno y fue un paso clave hacia la física cuántica moderna, construyendo sobre ideas de Planck y Rutherford.
'''

In [136]:
rouge_gemini_context = rouge.compute(
    predictions=[gemini_context.text],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_gemini_context = bleu.compute(
    predictions=[gemini_context.text],
    references=[chat_response],
)

rouge_gemini_base = rouge.compute(
    predictions=[gemini_base.text],
    references=[chat_response],
    use_aggregator=True,
    use_stemmer=True,
)

bleu_gemini_base = bleu.compute(
    predictions=[gemini_base.text],
    references=[chat_response],
)

In [137]:
compare_rouge(rouge_gemini_context, rouge_gemini_base, "GEMINI CONTEXTO", "GEMINI BASE")


🔎 Comparando ROUGE entre GEMINI CONTEXTO y GEMINI BASE

ROUGE1   | GEMINI CONTEXTO: 0.482 | GEMINI BASE: 0.495 → 🏆 GEMINI BASE
ROUGE2   | GEMINI CONTEXTO: 0.222 | GEMINI BASE: 0.306 → 🏆 GEMINI BASE
ROUGEL   | GEMINI CONTEXTO: 0.336 | GEMINI BASE: 0.343 → 🏆 GEMINI BASE
ROUGELSUM | GEMINI CONTEXTO: 0.365 | GEMINI BASE: 0.374 → 🏆 GEMINI BASE

📊 Promedio de métricas:
GEMINI CONTEXTO: 0.351
GEMINI BASE: 0.380

✅ GEMINI BASE tiene mejor solapamiento global con la referencia.


In [138]:
compare_bleu(bleu_gemini_context, bleu_gemini_base, "GEMINI CONTEXTO", "GEMINI BASE")


🔎 Comparando GEMINI CONTEXTO vs GEMINI BASE

GEMINI CONTEXTO BLEU: 0.11053882014042914
GEMINI BASE BLEU: 0.12925607807122994
Precisión 1-gram: GEMINI CONTEXTO=0.462 | GEMINI BASE=0.368
Precisión 2-gram: GEMINI CONTEXTO=0.172 | GEMINI BASE=0.181
Precisión 3-gram: GEMINI CONTEXTO=0.079 | GEMINI BASE=0.096
Precisión 4-gram: GEMINI CONTEXTO=0.032 | GEMINI BASE=0.044

Longitud traducción: GEMINI CONTEXTO=65 | GEMINI BASE=117
Referencia: 70 (idéntica para ambos)

Brevity penalty: GEMINI CONTEXTO=0.9260 | GEMINI BASE=1.0000

🏆 GEMINI CONTEXTO tiene mejores coincidencias de n-gramas.
📏 GEMINI BASE es más largo y cubre más de la referencia.


Podemos observar como las técnicas de RAG pueden no siempre resultar útiles, en especial en modelos con bases de datos superiores a las nuestras, esto lo podemos observar sobre como GEMINI BASE obtuvo una victoria según las métricas en este último enfoque de RAG